In [3]:
onedrive = r"C:\Users\Mike\OneDrive"

from boututils.datafile import DataFile
from boutdata.collect import collect
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os, sys, pathlib
import platform
import traceback
import xarray as xr
import xbout
from pathlib import Path
from boututils.datafile import DataFile

import shutil

sys.path.append(os.path.join(onedrive,r"Project\python-packages\sdtools"))

from hermes3.case_db import *
from hermes3.load import *
from hermes3.named_selections import *
from hermes3.plotting import *
from hermes3.grid_fields import *
from hermes3.accessors import *
from hermes3.utils import *

%load_ext autoreload
%autoreload 2


print("Done")

Done


In [14]:
constants("q_e")

1.60217662e-19

In [60]:
path_old = r"C:\Users\Mike\OneDrive\Project\collab\tech\grid\MAST-U\mu1af5-no_power_src.nc"
path = r"C:\Users\Mike\OneDrive\Project\collab\tech\grid\MAST-U\mu1af5-no_power_srcTEST.nc"
shutil.copy(path_old, path)

import netCDF4 as nc
class puff():
    def __init__(self, path):
        
        with DataFile(path) as mesh:
            self.path = path
            
            self.pressure_source = mesh["Pd_src"].squeeze()
            self.energy_source = self.pressure_source * 3/2
            self.density_source = mesh["Nd_src"].squeeze()

            self.dv = mesh["dv"]

            self.energy_flow = (self.pressure_source * self.dv).sum() * 3/2
            self.density_flow = (self.density_source * self.dv).sum()
            self.particle_energy = self.energy_flow / self.density_flow / 1.60217662e-19
            
            self.puff_volume = 0
            
            for i, dv in enumerate(self.dv.flatten()):
                if self.density_source.flatten()[i] > 0:
                    self.puff_volume += dv
            

    def read(self):
        
        print("-"*30)
        print("Flowrates")
        print("-"*30)
        
        print(f"Particle flow rate: {self.density_flow:.2e} [s-1]")
        print(f"Total power in: {self.energy_flow:.2e} [W]")
        print(f"Particle energy: {self.particle_energy:.2e} [eV]")
        
        print()
        
        print("-"*30)
        print("Sources")
        print("-"*30)
        
        print(f"Density source: {self.density_source.mean():.2e} [m-3 s-1]")
        print(f"Pressure source: {self.pressure_source.mean():.2e} [Pa s^-1 m-3]")
        print(f"Energy source: {self.energy_source.mean():.2e} [W m-3]")
        
        
        
    def update(self, density_flow_new):
        
        density_source_new_scalar = density_flow_new / self.puff_volume
        energy_source_new_scalar = density_flow_new * self.particle_energy * 1.60217662e-19 / self.puff_volume
        pressure_source_new_scalar = energy_source_new_scalar * 2/3
        
        print()
        print("-"*30)
        print("Updating")
        print("-"*30)
        print(f"New flow rate: {density_flow_new:.2e} [s-1]")
        print(f"New density source: {density_source_new_scalar:.2e} [m-3 s-1]")

        
        with nc.Dataset(self.path, "r+") as ds:
            
            for x in range(self.density_source.shape[0]):
                for y in range(self.density_source.shape[1]):
                    
                    if ds["Nd_src"][x,y] > 0:
                        ds["Nd_src"][x,y] = density_source_new_scalar
                        ds["Pd_src"][x,y] = pressure_source_new_scalar
        
                    

        
    

p = puff(path)
p.read()
print("\n","O"*50)
p.update(3e21)
print("\n","O"*50)
p.read()

------------------------------
Flowrates
------------------------------
Particle flow rate: 1.50e+21 [s-1]
Total power in: 7.21e+02 [W]
Particle energy: 3.00e+00 [eV]

------------------------------
Sources
------------------------------
Density source: 6.50e+20 [m-3 s-1]
Pressure source: 2.08e+02 [Pa s^-1 m-3]
Energy source: 3.13e+02 [W m-3]

 OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO

------------------------------
Updating
------------------------------
New flow rate: 3.00e+21 [s-1]
New density source: 2.81e+24 [m-3 s-1]

 OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO
------------------------------
Flowrates
------------------------------
Particle flow rate: 1.50e+21 [s-1]
Total power in: 7.21e+02 [W]
Particle energy: 3.00e+00 [eV]

------------------------------
Sources
------------------------------
Density source: 6.50e+20 [m-3 s-1]
Pressure source: 2.08e+02 [Pa s^-1 m-3]
Energy source: 3.13e+02 [W m-3]


In [37]:
p.density_source.shape

(40, 108)

In [27]:
x = p.dv.flatten()

for i in x:
    print(i)

4.388927306161333e-05
5.113600053143644e-05
5.9495910836946934e-05
6.910827703987416e-05
8.061310021271991e-05
9.434334415337014e-05
0.00011131912661033471
0.00013280695015731883
0.00016033396523161076
0.0001974765541625674
0.00024280693992252456
0.00028929298858281535
0.0015905248357366605
0.002710963587498633
0.004117216944934133
0.00478233782895656
0.004574459244351604
0.00404924149175285
0.0035382596425832364
0.0031365674515501695
0.0028630230713573585
0.002724155910427625
0.002724155922336245
0.002863023579003983
0.00313656795039599
0.0035382595190209987
0.00404924112550758
0.004574461248531739
0.004782339867722825
0.004117217205137447
0.0027109696895066425
0.0015905189968822263
0.0002892925788098269
0.00024280732111870234
0.00019747645151000238
0.00016033389511926213
0.00013280698014999292
0.00011131912118529122
9.434333394255444e-05
8.061312422776614e-05
6.910834174661922e-05
5.949588005914459e-05
5.326862880474931e-05
4.94022288417329e-05
0.00041436037468096774
0.00068739748590